In [1]:
import pandas as pd

df = pd.read_csv('data/NiftyNext50Scrips.csv')

stock_ids = df['Symbol']


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

ACC
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00

[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DMART.NS: Data doesn't exist for startDate = 1644431400, endDate = 1644863400
BAJAJHLDNG
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:0

164
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:

[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
HINDPETRO
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[******

[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
JINDALSTEL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
PIDILITIND
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
MCDOWELL-N
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*****

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,ACC.NS,2020-08-15,1385.30,14.43,2602.0,62.29,13692.0,22.24
7,ACC.NS,2020-11-15,1668.05,19.38,3537.0,65.56,13701.0,25.44
8,ACC.NS,2021-02-15,1765.75,25.16,4145.0,76.17,13786.0,23.18
9,ACC.NS,2021-05-16,1876.80,29.96,4292.0,88.93,14576.0,21.10
10,ACC.NS,2021-08-15,2308.60,30.32,3885.0,104.82,15859.0,22.02
...,...,...,...,...,...,...,...,...
7,YESBANK.NS,2020-11-15,14.55,0.05,3271.0,-70.59,15372.0,-0.21
8,YESBANK.NS,2021-02-15,16.40,0.06,2760.0,2.26,13555.0,7.26
9,YESBANK.NS,2021-05-16,13.20,-1.51,3003.0,-1.37,12611.0,-9.64
10,YESBANK.NS,2021-08-15,11.90,0.08,3123.0,-1.32,12157.0,-9.02


In [3]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-3-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-3-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-3-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,ACC.NS,2020-08-15,1385.30,14.43,2602.0,62.29,13692.0,22.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ACC.NS,2020-11-15,1668.05,19.38,3537.0,65.56,13701.0,25.44,14.43,62.29,2602.0,22.24,1385.30,5.25,15.14,20.39,35.93,34.30,20.39
8,ACC.NS,2021-02-15,1765.75,25.16,4145.0,76.17,13786.0,23.18,19.38,65.56,3537.0,25.44,1668.05,16.18,-10.32,5.86,17.19,29.82,5.86
9,ACC.NS,2021-05-16,1876.80,29.96,4292.0,88.93,14576.0,21.10,25.16,76.17,4145.0,23.18,1765.75,16.75,-10.48,6.27,3.55,19.08,6.27
10,ACC.NS,2021-08-15,2308.60,30.32,3885.0,104.82,15859.0,22.02,29.96,88.93,4292.0,21.10,1876.80,17.87,5.14,23.01,-9.48,1.20,23.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,YESBANK.NS,2020-11-15,14.55,0.05,3271.0,-70.59,15372.0,-0.21,0.03,-73.11,3577.0,-0.21,15.10,-3.45,-0.00,-3.45,-8.55,66.67,-3.45
8,YESBANK.NS,2021-02-15,16.40,0.06,2760.0,2.26,13555.0,7.26,0.05,-70.59,3271.0,-0.21,14.55,-103.20,116.03,12.83,-15.62,20.00,12.83
9,YESBANK.NS,2021-05-16,13.20,-1.51,3003.0,-1.37,12611.0,-9.64,0.06,2.26,2760.0,7.26,16.40,-160.62,141.18,-19.44,8.80,-2616.67,-19.44
10,YESBANK.NS,2021-08-15,11.90,0.08,3123.0,-1.32,12157.0,-9.02,-1.51,-1.37,3003.0,-9.64,13.20,-3.65,-6.20,-9.85,4.00,-105.30,-9.85


In [4]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,INDIGO.NS,2021-11-15,86.50,-54.80,37.50
10,DMART.NS,2021-11-15,50.28,338.78,41.29
11,BAJAJHLDNG.NS,2021-11-15,50.00,45.33,18.76
11,SIEMENS.NS,2021-11-15,46.42,131.54,6.23
11,IGL.NS,2021-11-15,45.66,50.63,-7.02
11,MCDOWELL-N.NS,2021-11-15,45.64,428.95,45.13
11,ICICIPRULI.NS,2021-11-15,42.87,-340.31,0.18
11,PIDILITIND.NS,2021-11-15,35.57,70.21,9.94
11,PGHH.NS,2021-11-15,34.43,345.66,15.52
11,ADANIGREEN.NS,2021-11-15,32.55,-54.29,40.60


In [5]:
df_growth[df_growth['Stock'] == 'ADANIGREEN.NS']

,Stock,Quarter,Sales%,EPS%,Return%
6,ADANIGREEN.NS,2020-08-15,NaN,NaN,NaN
7,ADANIGREEN.NS,2020-11-15,-21.34,-58.62,182.05
8,ADANIGREEN.NS,2021-02-15,22.39,133.33,5.98
9,ADANIGREEN.NS,2021-05-16,31.64,139.29,1.79
10,ADANIGREEN.NS,2021-08-15,-0.91,108.96,-15.53
11,ADANIGREEN.NS,2021-11-15,32.55,-54.29,40.60


In [6]:
df_analysis_final.to_csv('data/NiftyNext_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_growth = pd.read_csv('data/NiftyNext_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,ACC.NS,2020-11-15,1668.05,19.38,3537.0,65.56,13701.0,25.44,14.43,62.29,2602.0,22.24,1385.30,5.25,15.14,20.39,35.93,34.30,20.39
2,ACC.NS,2021-02-15,1765.75,25.16,4145.0,76.17,13786.0,23.18,19.38,65.56,3537.0,25.44,1668.05,16.18,-10.32,5.86,17.19,29.82,5.86
3,ACC.NS,2021-05-16,1876.80,29.96,4292.0,88.93,14576.0,21.10,25.16,76.17,4145.0,23.18,1765.75,16.75,-10.48,6.27,3.55,19.08,6.27
4,ACC.NS,2021-08-15,2308.60,30.32,3885.0,104.82,15859.0,22.02,29.96,88.93,4292.0,21.10,1876.80,17.87,5.14,23.01,-9.48,1.20,23.01
5,ACC.NS,2021-11-15,2571.95,23.97,3749.0,109.41,16071.0,23.51,30.32,104.82,3885.0,22.02,2308.60,4.38,7.06,11.44,-3.50,-20.94,11.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,YESBANK.NS,2020-11-15,14.55,0.05,3271.0,-70.59,15372.0,-0.21,0.03,-73.11,3577.0,-0.21,15.10,-3.45,-0.00,-3.45,-8.55,66.67,-3.45
293,YESBANK.NS,2021-02-15,16.40,0.06,2760.0,2.26,13555.0,7.26,0.05,-70.59,3271.0,-0.21,14.55,-103.20,116.03,12.83,-15.62,20.00,12.83
294,YESBANK.NS,2021-05-16,13.20,-1.51,3003.0,-1.37,12611.0,-9.64,0.06,2.26,2760.0,7.26,16.40,-160.62,141.18,-19.44,8.80,-2616.67,-19.44
295,YESBANK.NS,2021-08-15,11.90,0.08,3123.0,-1.32,12157.0,-9.02,-1.51,-1.37,3003.0,-9.64,13.20,-3.65,-6.20,-9.85,4.00,-105.30,-9.85


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()

DABUR.NS         3
HAVELLS.NS       3
SAIL.NS          3
PIDILITIND.NS    3
VEDL.NS          3
DMART.NS         3
BERGEPAINT.NS    3
JINDALSTEL.NS    3
GODREJCP.NS      3
NMDC.NS          3
GAIL.NS          3
ADANIGREEN.NS    2
AMBUJACEM.NS     2
PGHH.NS          2
BAJAJHLDNG.NS    2
HDFCAMC.NS       2
JUBLFOOD.NS      2
BOSCHLTD.NS      2
IGL.NS           2
MUTHOOTFIN.NS    2
SIEMENS.NS       2
ACC.NS           2
MCDOWELL-N.NS    2
DLF.NS           2
PIIND.NS         2
LTI.NS           2
COLPAL.NS        2
MARICO.NS        2
LUPIN.NS         2
HINDPETRO.NS     2
NAUKRI.NS        2
SBICARD.NS       1
APOLLOHOSP.NS    1
ADANIENT.NS      1
BANDHANBNK.NS    1
INDUSTOWER.NS    1
PEL.NS           1
ADANITRANS.NS    1
INDIGO.NS        1
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
200,LTI.NS,2021-11-15,8.81,10.88,48.97,0,0,0.0,4.0,4.0
119,GAIL.NS,2021-11-15,23.83,34.93,-0.30,0,0,0.0,4.0,4.0
290,VEDL.NS,2021-11-15,7.00,9.33,-0.12,0,0,0.0,5.0,5.0
272,SAIL.NS,2021-11-15,29.96,11.23,-12.47,0,0,4.0,1.0,5.0
113,DABUR.NS,2021-11-15,7.89,15.38,2.41,0,0,3.0,3.0,6.0
128,GODREJCP.NS,2021-11-15,9.33,15.56,-6.47,0,0,3.0,3.0,6.0
188,JINDALSTEL.NS,2021-11-15,28.29,6078.05,-6.17,0,0,0.0,7.0,7.0
176,NAUKRI.NS,2021-11-15,11.31,4738.29,18.12,0,0,1.0,7.0,8.0
230,PIIND.NS,2021-11-15,13.40,22.61,-12.85,0,0,4.0,5.0,9.0
164,IGL.NS,2021-11-15,45.66,50.63,-7.02,0,0,4.0,7.0,11.0


In [12]:
df_growth[df_growth['Stock'] == 'LTI.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
196,LTI.NS,2020-11-15,1.66,9.11,24.52
197,LTI.NS,2021-02-15,5.17,13.79,32.97
198,LTI.NS,2021-05-16,3.68,5.05,-10.93
199,LTI.NS,2021-08-15,5.90,-9.01,35.14
200,LTI.NS,2021-11-15,8.81,10.88,48.97
